In [19]:
"""
Udacity Final Project
"""
import math
from tabulate import tabulate
# Given data
# Unique cookies to view course overview page per day
n_pageviews_perday = 40000
# Unique cookies to click "Start free trial" per day
n_clicks_perday = 3200
# Enrollments per day
enrollments_perday = 660
# Click-through-probability on "Start free trial"
ctp = 0.08
# Probability of enrolling: enroll/click
gross_conversion = 0.20625
# Probability of payment: payment/enroll
retention = 0.53
# Probability of payment: payment/click
net_conversion = 0.1093125
# Number of cookies visiting the course overview page
n_sample = 5000


SD of GC:  0.0202
SD of Retention:  0.0549
SD of NC:  0.0156
Size of GC: 645875.0
Size of Retention: 4741212.121212121
Size of NC: 685325.0
Duration: 118.53030303030303
New Duration: 17.133125
Pageviews prob: 0.5006396668806133
Clicks prob: 0.5004673474066628
Pageviews SE: 0.0006
Clicks SE: 0.0021
Pageviews ME: 0.0012
Clicks ME: 0.0041
Pageviews Interval [ 0.4988, 0.5012 ]
Clicks Interval [ 0.4959, 0.5041 ]
Lower bound of Number of Cookies/Pageviews: 0.4988
Upper bound of Number of Cookies/Pageviews: 0.5012
Lower bound of Number of Clicks: 0.4959
Upper bound of Number of Clicks: 0.5041
0.08212581357457682
0.08218244066616376
5.662709158693602e-05
0.0006610610775037591
CTP Interval [ -0.0013, 0.0013 ]
------------------  -------  ----------
/                   Control  Experiment
Clicks              17293    17260
Enrollments         3785     3423
Payments            2033     1945
Probability_GC      0.21887  0.19831
Probability_NC      0.11756  0.11268
Pooled_Probability  0.2086   0.11

### Calculate Standard Deviation

In [ ]:
# gross_conversion
std_gross_conversion=math.sqrt(gross_conversion*(1-gross_conversion)/(n_clicks_perday/n_pageviews_perday * n_sample))
# retention
std_retention=math.sqrt(retention*(1-retention)/(enrollments_perday/n_pageviews_perday*n_sample))
# net_conversion
std_net_conversion=math.sqrt(net_conversion*(1-net_conversion)/(n_clicks_perday/n_pageviews_perday*n_sample))
print("SD of GC: ", round(std_gross_conversion, 4))
print("SD of Retention: ", round(std_retention, 4))
print("SD of NC: ", round(std_net_conversion, 4))

### Sizing

In [ ]:
# Using Sample Size Calculator https://www.evanmiller.org/ab-testing/sample-size.html
# Baseline conversion rate is the probability
# Minimum Detectable Effect is 1%

# Gross Conversion_sample size_per group = 25835
print("Size of GC:", 25835 * 2 / ctp)
# Retention_sample size_per group = 39115
print("Size of Retention:", 39115 * 2 / (enrollments_perday / n_pageviews_perday))
size = 39115 * 2 / (enrollments_perday / n_pageviews_perday)
# Net Conversion_sample size_per group = 27413
print("Size of NC:", 27413 * 2 / ctp)
# Take the maximum number as the sample size of the experiment, 4741212
size1 = 27413 * 2 / ctp
# Duration and exposure
print("Duration:", size / n_pageviews_perday) # 119 days, too long
print("New Duration:", size1 / n_pageviews_perday) # 17 days

### Sanity Check

In [ ]:
# Sum of Pageviews in Control group, 345543
# Sum of Clicks in Control group, 28378
# Sum of Pageviews in Experiment group, 344660
# Sum of Clicks in Experiment group, 28325
# Probability of Control/Total
prob_p = 345543 / (345543 + 344660)
print("Pageviews prob:", prob_p) # around 0.5, which represents invariance
prob_c = 28378 / (28378 + 28325)
print("Clicks prob:", prob_c) # around 0.5
# Standard Error
se_p = round(math.sqrt(0.5 * (1 - 0.5) / (345543 + 344660)), 4)
se_c = round(math.sqrt(0.5 * (1 - 0.5) / (28378 + 28325)), 4)
print("Pageviews SE:", se_p)
print("Clicks SE:", se_c)
# Margin of Error
me_p = round(1.96 * se_p, 4)
me_c = round(1.96 * se_c, 4)
print("Pageviews ME:", me_p)
print("Clicks ME:", me_c)
# Lower and Upper bounds
print(f"Pageviews Interval [ {0.5- me_p}, {0.5 + me_p} ]" )
print(f"Clicks Interval [ {0.5 - me_c}, {0.5 + me_c} ]")
print("Lower bound of Number of Cookies/Pageviews:", 0.5 - me_p)
print("Upper bound of Number of Cookies/Pageviews:", 0.5 + me_p)

print("Lower bound of Number of Clicks:", 0.5 - me_c)
print("Upper bound of Number of Clicks:", 0.5 + me_c)

# Difference of Click-through-probability between two groups
ctp_con = 28378 / 345543
ctp_exp = 28325 / 344660
p_diff = abs(ctp_con - ctp_exp)
p_exp = 0 # Expect to be zero to pass sanity test of CTP
print(ctp_con)
print(ctp_exp)
print(p_diff)
se_ctp = math.sqrt(ctp_con * (1 - ctp_con) / 345543 + ctp_exp * (1 - ctp_exp) / 344660)
print(se_ctp)
me_ctp = round(1.96 * se_ctp, 4)
print(f"CTP Interval [ {0 - me_ctp}, {0 + me_ctp} ]")


### Effect Size Tests

In [27]:
# Reorganize the data
table = [['/' , 'Control', 'Experiment'], ['Clicks', 17293, 17260], ['Enrollments', 3785, 3423], ['Payments', 2033, 1945],
         ['Probability_GC', 0.21887, 0.19831], ['Probability_NC', 0.11756, 0.11268], ['Pooled_Probability', 0.20860, 0.11512],
         #['Standard Error', gc_se_con, gc_se_exp]
        ]
print(tabulate(table))

# Gross Conversion, enroll/click
d_min = 0.01
gc_con = 0.2189  # observed
gc_exp = 0.1983
gc_diff = gc_exp - gc_con
p_pool = (3785 + 3423) / (17293 + 17260)
gc_se_pool = math.sqrt(p_pool * (1 - p_pool) * (1/17293 + 1/17260)) # Be careful of the formula! 
gc_me = gc_se_pool * 1.96
print(f"GC Interval [ {gc_diff - gc_me}, {gc_diff + gc_me} ]")
print(f"Statistically significant is {gc_diff + gc_me < 0 or gc_diff - gc_me > 0} as CI does not include 0.")
print(f"Practically significant is {gc_diff + gc_me < d_min or gc_diff - gc_me > d_min} as CI does not include practical significance boundary." )


------------------  -------  ----------
/                   Control  Experiment
Clicks              17293    17260
Enrollments         3785     3423
Payments            2033     1945
Probability_GC      0.21887  0.19831
Probability_NC      0.11756  0.11268
Pooled_Probability  0.2086   0.11512
------------------  -------  ----------
GC Interval [ -0.029168483755042843, -0.012031516244957172 ]
Statistically significant is True as CI does not include 0.
Practically significant is True as CI does not include practical significance boundary.


In [33]:
# Retention, payment/enroll

d_min = 0.01
rt_con = 2033/3785
rt_exp = 1945/3423
rt_diff = rt_exp - rt_con
p_pool_retention = (2033 + 1945) / (3785 + 3423)
rt_se_pool = math.sqrt(p_pool_retention * (1 - p_pool_retention) * (1/3785 + 1/3423))
rt_me = rt_se_pool * 1.96
print(f"RT Interval [ {rt_diff - rt_me}, {rt_diff + rt_me} ]")
print(f"Statistically significant is {rt_diff + rt_me < 0 or rt_diff - rt_me > 0} as CI does not include 0.")
print(f"Practically significant is {rt_diff - rt_me > d_min} as CI includes practical significance boundary." )


RT Interval [ 0.008104435728019967, 0.05408517368626556 ]
Statistically significant is True as CI does not include 0.
Practically significant is False as CI includes practical significance boundary.


In [34]:
# Net conversion, payment/click

d_min = 0.0075
nc_con = 2033/17293
nc_exp = 1945/17260
nc_diff = nc_exp - nc_con
p_pool_nc = (2033 + 1945) / (17293 + 17260)
nc_se_pool = math.sqrt(p_pool_nc * (1 - p_pool_nc) * (1/17293 + 1/17260))
nc_me = nc_se_pool * 1.96
print(f"RT Interval [ {nc_diff - nc_me}, {nc_diff + nc_me} ]")
print(f"Statistically significant is {nc_diff + nc_me < 0 or nc_diff - nc_me > 0} as CI does not include 0.")
print(f"Practically significant is {nc_diff - nc_me > d_min} as CI includes practical significance boundary." )


RT Interval [ -0.011604624359891718, 0.001857179010803383 ]
Statistically significant is False as CI does not include 0.
Practically significant is False as CI includes practical significance boundary.


### Effect Size Tests

In [ ]:
from scipy.stats import binom_test

# Gross Conversion
alpha = 0.05
beta = 0.2
p = 0.5

# Calculate the differences of gc rate (enroll/clicks) 
# Count the number of positive differences, 4/23
# Use Sign and binomial test https://www.graphpad.com/quickcalcs/binomial2/
# two-tail P value is 0.0026
# As p value is smaller than alpha, it's statistically significant.

In [35]:
# Retention
alpha = 0.05
beta = 0.2
p = 0.5

# Calculate the differences of retention rate (payment/enroll) 
# Count the number of positive differences, 13/23
# Use Sign and binomial test https://www.graphpad.com/quickcalcs/binomial2/
# two-tail P value is 0.6776
# As p value is greater than alpha, it's NOT statistically significant.

[('Java', 14), ('Python', 3), ('JavaScript', 6)]


In [ ]:
# Net conversion, payment/click
alpha = 0.05
beta = 0.2
p = 0.5

# Calculate the differences of net conversion rate (payment/click) 
# Count the number of positive differences, 10/23
# Use Sign and binomial test https://www.graphpad.com/quickcalcs/binomial2/
# two-tail P value is 0.6776
# As p value is greater than alpha, it's NOT statistically significant.